<a href="https://colab.research.google.com/github/mitramir55/PassivePy/blob/main/PassivePy%20notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# first run this cell to install the package


!pip install -r https://raw.githubusercontent.com/mitramir55/PassivePy/main/PassivePyCode/PassivePySrc/requirements_lg.txt
!pip install -i https://test.pypi.org/simple/ PassivePy==0.0.58

  Using cached en_core_web_lg-3.0.0-py3-none-any.whl
Looking in indexes: https://test.pypi.org/simple/


In [2]:
# then run this to import the package and create PassivePy
import pandas as pd
import os
import regex as re
pd.set_option('display.max_colwidth', None)
import string
from PassivePySrc import PassivePy


spacy_model = "en_core_web_lg"
passivepy = PassivePy.PassivePyAnalyzer(spacy_model)

## Single sentence example

In [6]:
# Try changing the sentence below:
sample_text = "Natural resources are exhausted by humans."
resutl_1 = passivepy.match_text(sample_text, full_passive=True, truncated_passive=True)
resutl_1

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

,document,count_sents,all_passives,passive_count,passive_sents_count,passive_percentages,binary,full_passive_matches,full_passive_count,full_passive_sents_count,full_passive_percentages,binary_full_passive,truncated_passive_matches,truncated_passive_count,truncated_passive_sents_count,truncated_passive_percentages,binary_truncated_passive
0,Natural resources are exhausted by humans.,1,[exhausted by],1,1,1,1,[exhausted by],1,1,1,1,[],0,0,0,0


In [7]:
# parse each senence 
# POS: coarse-grained part of speech tag
# dependency: dependency tag
# tag: fine-grained part of speech tags
# lemma: canonical form of the word


passivepy.parse_sentence(sample_text)

,Natural,resources,are,exhausted,by,humans,.
POS,ADJ,NOUN,AUX,VERB,ADP,NOUN,PUNCT
dependency,amod,nsubjpass,auxpass,ROOT,agent,pobj,punct
tag,JJ,NNS,VBP,VBN,IN,NNS,.
lemma,natural,resource,be,exhaust,by,human,.


In [ ]:
# import a dataset from your colab

# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
# or use the sample dataset in our github

!git clone https://github.com/mitramir55/PassivePy.git

Cloning into 'PassivePy'...
remote: Enumerating objects: 872, done.
remote: Counting objects: 100% (872/872), done.
remote: Compressing objects: 100% (587/587), done.
remote: Total 872 (delta 433), reused 662 (delta 234), pack-reused 0
Receiving objects: 100% (872/872), 865.59 KiB | 11.39 MiB/s, done.
Resolving deltas: 100% (433/433), done.


In [13]:
df = pd.read_excel('/content/PassivePy/Data/passivepy_humancoding_match.xlsx')
df.drop(columns=['Unnamed: 0', 'PassiveMatch(es)', 'PassiveCount', 'match'], inplace=True)
df.head(1)

,Sentence,ID,Test,Cond,human_coding
0,The politics being discussed were causing scene.,A3ATU3VULAB7K2,2.0,2,1


## Sentence-level from a dataset

In [15]:
# with sentence-level analysis, we see each record as a document 
# that needs to be separated into sentences and then be analyzed.


df_detected_s = passivepy.match_sentence_level(df, column_name='Sentence', n_process = 1,
                                                batch_size = 1000, add_other_columns=True,
                                                truncated_passive=False, full_passive=False)
df_detected_s

Detecting Sentences...


  0%|          | 0/1150 [00:00<?, ?it/s]

Total number of sentences = 1156
Starting to find passives...


  0%|          | 0/1156 [00:00<?, ?it/s]

,docId,sentenceId,sentences,all_passives,all_passives_count,binary,ID,Test,Cond,human_coding
0,1,1,the politics being discussed were causing scene.,[being discussed],1,1,A3ATU3VULAB7K2,2.0,2,1
1,2,1,the world of politics is quite fascinating.,None,0,0,A1CTJ8UT7SF317,1.0,1,0
2,3,1,i don usually participate in discussions about politics.,None,0,0,A327Q0Y175MEH7,1.0,1,0
3,4,1,the president was impeached before his second term.,[was impeached],1,1,A19TD2J8506A4Y,1.0,2,1
4,5,1,the president just gave speech about covid relief.,None,0,0,A2DDXYTEVDV5F5,1.0,1,0
...,...,...,...,...,...,...,...,...,...,...
1151,1146,1,the shareholders were not satisfied by the ceo explanations,None,0,0,A1EDKVYT9ZCGUH,2.0,2,1
1152,1147,1,i would mind what you are doing in this meeting.,None,0,0,A3CX8S6Q0QEW8E,1.0,1,0
1153,1148,1,i had no business doing that.,None,0,0,AYX9OCFWBM939,2.0,2,0
1154,1149,1,they had good business sales then.,None,0,0,A1HNB2243A2PQH,2.0,2,0


## Corpus level


In [17]:
df_detected_c = passivepy.match_corpus_level(df, column_name='Sentence', n_process = 1,
                                            batch_size = 1000, add_other_columns=True,
                                            truncated_passive=False, full_passive=False)
df_detected_c

Detecting Sentences...


  0%|          | 0/1150 [00:00<?, ?it/s]

Total number of sentences = 1156
Starting to find passives...


  0%|          | 0/1156 [00:00<?, ?it/s]

  0%|          | 0/1150 [00:00<?, ?it/s]

,document,count_sents,all_passives,passive_count,passive_sents_count,passive_percentages,binary,ID,Test,Cond,human_coding
0,The politics being discussed were causing scene.,1,[being discussed],1,1,1,1,A3ATU3VULAB7K2,2.0,2,1
1,The world of politics is quite fascinating.,1,[],0,0,0,0,A1CTJ8UT7SF317,1.0,1,0
2,I don usually participate in discussions about politics.,1,[],0,0,0,0,A327Q0Y175MEH7,1.0,1,0
3,The president was impeached before his second term.,1,[was impeached],1,1,1,1,A19TD2J8506A4Y,1.0,2,1
4,The President just gave speech about COVID relief.,1,[],0,0,0,0,A2DDXYTEVDV5F5,1.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
1145,The shareholders were not satisfied by the CEO explanations,1,[],0,0,0,0,A1EDKVYT9ZCGUH,2.0,2,1
1146,I would mind what you are doing in this meeting.,1,[],0,0,0,0,A3CX8S6Q0QEW8E,1.0,1,0
1147,I had no business doing that.,1,[],0,0,0,0,AYX9OCFWBM939,2.0,2,0
1148,They had good business sales then.,1,[],0,0,0,0,A1HNB2243A2PQH,2.0,2,0


In [18]:
len(df_detected_c[df_detected_c['human_coding']==df_detected_c['binary']])/len(df_detected_c) # accuracy in this case

0.9791304347826087